<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_Semantica_Vectoria_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantica Vectorial para procesar sentencias

In [1]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json

In [2]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [3]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [4]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


# Lectura de tablas de datos primarios

In [5]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

In [ ]:
tbfallos['textos_fallo'][1]

In [6]:
tbfallos.head()

,textos_fallo,nro_causa
0,#TEXTO COMPLETO#A C U E R D O#En la ciudad de ...,122176
1,#TEXTO COMPLETO#A C U E R D O#En la ciudad de ...,122321
2,"#TEXTO COMPLETO#""VOSS SUSANA BEATRIZ C/ VILLAR...",123033
3,#TEXTO COMPLETO#A C U E R D O#En la ciudad de ...,122514
4,"#TEXTO COMPLETO#""RICA ARIEL FEDERICO Y OTROS C...",123396


# Tokenizacion

In [ ]:
! pip install stanza

In [9]:
import stanza
stanza.download('es') # download spanish model

2020-10-26 11:46:09 INFO: Downloading default packages for language: es (Spanish)...
2020-10-26 11:52:53 INFO: Finished downloading models and saved to /root/stanza_resources.


In [103]:
fallos = tbfallos['textos_fallo'][1]
fallos = fallos.replace("#","\n\n")
#fallos = fallos.values.tolist()

In [106]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse', tokenize_no_ssplit=True)
doc = nlp(fallos)

2020-10-26 13:05:34 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2020-10-26 13:05:34 INFO: Use device: gpu
2020-10-26 13:05:34 INFO: Loading: tokenize
2020-10-26 13:05:34 INFO: Loading: mwt
2020-10-26 13:05:34 INFO: Loading: pos
2020-10-26 13:05:35 INFO: Loading: lemma
2020-10-26 13:05:35 INFO: Loading: depparse
2020-10-26 13:05:36 INFO: Loading: ner
2020-10-26 13:05:38 INFO: Done loading processors!


In [110]:
doc.sentences[0].text

'TEXTO COMPLETO'

In [ ]:
for sent in doc.sentences:
    print(sent.text)

In [ ]:
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [61]:
dicts = doc.to_dict() # dicts is List[List[Dict]], representing each token / word in each sentence in the document

# Manipulando objeto "doc"

In [63]:
def print_doc_info(doc):
    print(f"Num sentences:\t{len(doc.sentences)}")
    print(f"Num tokens:\t{doc.num_tokens}")
    print(f"Num words:\t{doc.num_words}")
    print(f"Num entities:\t{len(doc.entities)}")

In [85]:
print_doc_info(doc)

Num sentences:	47
Num tokens:	2008
Num words:	2013
Num entities:	92


In [65]:
def word_info_df(doc):
    """
    - Parameters: doc (a Stanza Document object)
    - Returns: A Pandas DataFrame object with one row for each token in
      doc, and columns for text, lemma, upos, and xpos.
    """
    rows = []
    for sentence in doc.sentences:
        for word in sentence.words:
            row = {
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
            }
            rows.append(row)
    return pd.DataFrame(rows)

In [74]:
word_info_df(doc)

,text,lemma,upos,xpos
0,TEXTO,texto,NOUN,NOUN
1,COMPLETO,completo,ADJ,ADJ
2,A,a,ADP,ADP
3,C,c,NOUN,NOUN
4,U,U,CCONJ,CCONJ
...,...,...,...,...
2008,KOGAN,KOGAN,PROPN,PROPN
2009,CARLOS,CARLOS,PROPN,PROPN
2010,E.,E.,PROPN,PROPN
2011,CAMPS,CAMPS,PROPN,PROPN


In [76]:
doc.num_words

2013

# Trabajando con las entidades

In [119]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

entity: TEXTO COMPLETO	type: MISC
entity: La Plata	type: LOC
entity: Acuerdo 2078	type: MISC
entity: Soria	type: PER
entity: Lázzari	type: PER
entity: Genoud	type: PER
entity: Kogan	type: PER
entity: Suprema Corte de Justicia	type: ORG
entity: C. 122.321	type: MISC
entity: Sala I de la Cámara de Apelación	type: ORG
entity: Civil	type: MISC
entity: Comercial de San Isidro	type: ORG
entity: señor R. F.	type: PER
entity: Curaduría Oficial de Alienados	type: ORG
entity: Asesoría de Incapaces n° 2	type: ORG
entity: Procurador General	type: PER
entity: Suprema Corte	type: ORG
entity: Juez doctor Soria	type: PER
entity: señor R. F.	type: PER
entity: Juzgado de Familia n° 4	type: ORG
entity: San Isidro	type: LOC
entity: Curaduría Oficial de Alienados de San Isidro	type: ORG
entity: Zárate-Campana	type: ORG
entity: Cuenta de Autos	type: MISC
entity: Cuenta Fiscal de Curaduría	type: ORG
entity: señor F.	type: PER
entity: Asesora de Incapaces	type: ORG
entity: Daniela Elizabeth Ferreyra	type: PER

In [96]:
# select person entities
def select_person_entities(doc):
    return [ent for ent in doc.entities if ent.type == "PER"]

In [97]:
def person_df(doc):
    """
    - Parameters: doc (a Stanza Document object)
    - Returns: A Pandas DataFrame with one row for each entity in doc
      that has a "PERSON" type, and and columns text, type, start_char.
    """
    rows = []
    persons = select_person_entities(doc)
    for person in persons:
        row = {
            "text": person.text,
            "type": person.type,
            "start_char": person.start_char,
            "end_char": person.end_char
        }
        rows.append(row)
    return pd.DataFrame(rows)


In [98]:
person_df(doc)

,text,type,start_char,end_char
0,Soria,PER,220,225
1,Lázzari,PER,230,237
2,Genoud,PER,239,245
3,Kogan,PER,247,252
4,señor R. F.,PER,653,664
5,Procurador General,PER,952,970
6,Juez doctor Soria,PER,1255,1272
7,señor R. F.,PER,1363,1374
8,señor F.,PER,1836,1844
9,Daniela Elizabeth Ferreyra,PER,2075,2101


# Bibliografia

- https://medium.com/@severinperez/exploring-literature-with-the-stanza-nlp-package-927d5b6556bf 
- https://colab.research.google.com/github/stanfordnlp/stanza/blob/master/demo/Stanza_CoreNLP_Interface.ipynb#scrollTo=ezEjc9LeV2Xs



# Semantica Vectorial

In [169]:
for i, sent in enumerate(doc.sentences):
  sent.print_tokens()

<Token id=1;words=[<Word id=1;text=TEXTO;lemma=texto;upos=NOUN;xpos=NOUN;feats=Gender=Masc|Number=Sing;head=0;deprel=root>]>
<Token id=2;words=[<Word id=2;text=COMPLETO;lemma=completo;upos=ADJ;xpos=ADJ;feats=Gender=Masc|Number=Sing;head=1;deprel=amod>]>
<Token id=1;words=[<Word id=1;text=A;lemma=a;upos=ADP;xpos=ADP;feats=AdpType=Prep;head=2;deprel=case>]>
<Token id=2;words=[<Word id=2;text=C;lemma=c;upos=NOUN;xpos=NOUN;head=0;deprel=root>]>
<Token id=3;words=[<Word id=3;text=U;lemma=U;upos=CCONJ;xpos=CCONJ;head=5;deprel=cc>]>
<Token id=4;words=[<Word id=4;text=E;lemma=e;upos=CCONJ;xpos=CCONJ;head=5;deprel=cc>]>
<Token id=5;words=[<Word id=5;text=R;lemma=R;upos=PROPN;xpos=PROPN;head=2;deprel=conj>]>
<Token id=6;words=[<Word id=6;text=D;lemma=D;upos=PROPN;xpos=PROPN;head=5;deprel=flat>]>
<Token id=7;words=[<Word id=7;text=O;lemma=o;upos=CCONJ;xpos=CCONJ;head=2;deprel=punct>]>
<Token id=1;words=[<Word id=1;text=En;lemma=en;upos=ADP;xpos=ADP;feats=AdpType=Prep;head=3;deprel=case>]>
<Token 

In [ ]:
# Iterate over all tokens in all sentences
for i, sent in enumerate(doc.sentences):    
    for t in sent.tokens:
        print(t.text)

In [ ]:
# Iterate over all words in all sentences
for i, sent in enumerate(doc.sentences):    
    for w in sent.words:
        print(w.text)

In [ ]:
# Iterate over all entities in all sentences
for i, sent in enumerate(doc.sentences):    
    for e in sent.entities:
        print(e.text)

# Word count Matrix of documents


In [170]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tbfallos['textos_fallo']

In [178]:
df = tbfallos['textos_fallo']

In [180]:
cv = CountVectorizer()
cv.fit(df)
results = cv.transform(df)

In [181]:
print(results.shape) # Sparse matrix

(914, 44226)


In [185]:
features = cv.get_feature_names()
df_res = pd.DataFrame(results.toarray(), columns=features)

In [189]:
df_res.head()

,00,000,0000,00000016,00000018,00000235,00000261,00000262,00000365,0000056,00000565,00000566,00000570,00000602,00000603,00000622,00000673,00000770,00000771,00000833,00001,00002772,00003413,00003417,00003907,00003908,00008571,00009,0001,000156,00018462,0002,000325,0006,000663,000969,0009867,001,001026,001133,...,ítem,ítems,íter,ñamandú,ñata,óbice,óbices,óbito,ómnibus,óntico,óptica,óptima,óptimas,óptimo,órbita,órbitas,órdenes,órgano,órganos,ósea,óseas,óseo,ótico,óvulos,últ,última,últimamente,últimas,últimaáratioádel,último,últimos,única,únicamente,únicas,único,únicos,útero,útil,útiles,útilmente
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
